In [2]:
import pandas as pd
import numpy as np
from webscraping_metacritic import get_metacritic
import psycopg2 as pg
connection_args = #{fill in}
from io import StringIO
from tqdm import tqdm
import re

In [ ]:
## if you had anything in the retry then I would run get_metacritic()
## on it again until i had a complete movie_details and film_cast_crew
## dataframe
retry_metacritic, scores_and_dates, movie_details, film_cast_crew = get_metacritic()

In [ ]:
film_cast_crew.head()

In [ ]:
movie_details.head()

In [ ]:
scores_and_dates.head()

In [ ]:
len(retry_metacritic)

In [ ]:
## rather than having multiple lines for one movie, we are putting all details in one row
## per movie to get ready to put into postgresql
new_columns = movie_details.label.unique()
tbl = pd.concat(
    [movie_details.set_index(['movie_id']).groupby('label')['data'].get_group(key) for key in new_columns], 
    axis=1
)
tbl.columns = new_columns
movie_details = tbl.reset_index()

In [ ]:
# removing spaces from column titles
movie_details.columns = movie_details.columns.str.replace(' ', '_')
# making runtime into float
movie_details.runtime = movie_details.runtime.str.replace('min', '', regex=False)
movie_details.runtime = movie_details.runtime.str.replace('s', '', regex=False)
movie_details.runtime = movie_details.runtime.str.strip().astype(float)

In [3]:
def fix_rating(row):
    """
    If the mpaa rating is NR then there isn't a description for the reason that it is rating that 
    or if it is foriegn of television it has no descriptor either. If the movie is MPAA 
    rated then this is to return only the rating or in worse case it returns "Rated (rating)"
    """
    if row.movie_rating == 'NR':
        return 'NR'
    else:
        return ' '.join(row.movie_rating.split(' ')[:10]).split('  ')[0].split(' ')[0]

In [ ]:
mpaa_rating = movie_details.apply(lambda x: fix_rating(x), axis=1)
movie_details.movie_rating = mpaa_rating

In [ ]:
# making MPAA ratings so there won't be different values for the same rating
movie_details.movie_rating = movie_details.movie_rating.str.replace('Not Rated', 'NR', regex=False)
movie_details.movie_rating = movie_details.movie_rating.str.replace('Unrated', 'NR', regex=False)
movie_details.movie_rating = movie_details.movie_rating.str.replace('PG 13', 'PG-13', regex=False)
movie_details.movie_rating = movie_details.movie_rating.str.replace('PG - 13', 'PG-13', regex=False)
movie_details.movie_rating = movie_details.movie_rating.str.replace('RatPG-13', 'PG-13', regex=False)
movie_details.movie_rating = movie_details.movie_rating.str.replace('PG- 13', 'PG-13', regex=False)
movie_details.movie_rating = movie_details.movie_rating.str.replace(':', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('Rated', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('for', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('rated', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('Rating', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('(', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('surrendered;', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('Rate', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('Symbol', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.fillna('NR')
movie_details.movie_rating = movie_details.movie_rating.str.replace('-', '').str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.strip()

## changing TV and foriegn ratings into MPAA equivalents
movie_details.movie_rating = movie_details.movie_rating.str.replace('Not Rated', 'NR', regex=False)
movie_details.movie_rating = movie_details.movie_rating.str.replace('Unrated', 'NR', regex=False)
movie_details.movie_rating = movie_details.movie_rating.str.replace('PG 13', 'PG-13', regex=False)
movie_details.movie_rating = movie_details.movie_rating.str.replace('PG - 13', 'PG-13', regex=False)
movie_details.movie_rating = movie_details.movie_rating.str.replace('RatPG-13', 'PG-13', regex=False)
movie_details.movie_rating = movie_details.movie_rating.str.replace('PG- 13', 'PG-13', regex=False)
movie_details.movie_rating = movie_details.movie_rating.str.replace(':', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('Rated', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('for', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('rated', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('Rating', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('(', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('surrendered;', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('Rate', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.replace('Symbol', '', regex=False).str.strip()
movie_details.movie_rating = movie_details.movie_rating.fillna('NR')
movie_details.movie_rating = movie_details.movie_rating.str.replace('-', '').str.strip()
movie_details.movie_rating = movie_details.movie_rating.str.strip()

# manually rating movies that didn't have their rating on metacritic 
movie_details.iloc[10385,3] = 'R'
movie_details.iloc[10902,3] = 'R'
movie_details.iloc[9942,3] = 'PG13'
movie_details.iloc[13089,3] = 'PG13'
movie_details.iloc[1122,3] = 'R'
movie_details.iloc[4454,3] = 'PG13'
movie_details.iloc[10037,3] = 'PG'
movie_details.iloc[10903,3] = 'R'
movie_details.iloc[12494,3] = 'PG13'
movie_details.iloc[13349,3] = 'PG'
movie_details.iloc[1546,3] = 'R'
movie_details.iloc[6116,3] = 'R'
movie_details.iloc[9631,3] = 'R'
movie_details.iloc[12016,3] = 'R'
movie_details.iloc[1745,3] = 'R'
movie_details.iloc[1922,3] = 'PG13'
movie_details.iloc[4763,3] = 'PG13'
movie_details.iloc[8877,3] = 'PG13'
movie_details.iloc[10601,3] = 'PG13'
movie_details.iloc[12693,3] = 'PG13'
movie_details.iloc[5969,3] = 'PG13'
movie_details.iloc[10229,3] = 'PG13'
movie_details.iloc[2696,3] = 'PG13'
movie_details.iloc[8188,3] = 'PG13'
movie_details.iloc[4320,3] = 'PG'
movie_details.iloc[9545,3] = 'PG'
movie_details.iloc[4401,3] = 'NR'
movie_details.iloc[12218,3] = 'NR'
movie_details.iloc[11277,3] = 'PG13'
movie_details.iloc[7275,3] = 'PG'
movie_details.iloc[12769,3] = 'PG13'
movie_details.iloc[6765,3] = 'G'
movie_details.iloc[3026,3] = 'R'
movie_details.iloc[2458,3] = 'PG'
movie_details.iloc[11891,3] = 'PG13'
movie_details.iloc[3415,3] = 'R'

In [ ]:
movie_details_pg = movie_details.drop(columns=['home_release_date', 'official_url'])

In [ ]:
final = film_cast_crew

In [ ]:
final = final.drop_duplicates(keep='first').sort_values('movie_id').reset_index(drop=True)
final.data = final.data.str.replace('  ', ' ').str.strip()

In [ ]:
## changing all names that have more than one webpage per a person. ex: Lil Wanye has one 
## metacritic site for his credits but he is represented Lil Wanye and Lil' Wanye 
final.data = final.data.str.replace('Yi Ching', 'Yi-Ching')
final.data = final.data.str.replace('Lil Wayne', "Lil' Wanye")
final.data = final.data.str.replace('Chô', 'Cho')
final.data = final.data.str.replace('Dong Young', 'Dong-Young')
final.data = final.data.str.replace('Maria Fahl Vikander', 'Maria Fahl-Vikander', regex=False)
final.data = final.data.str.replace('Ciarán Owens', 'Ciaran Owens', regex=False)
final.data = final.data.str.replace('Deray', 'DeRay', regex=False)
final.data = final.data.str.replace('Ken H Keller', 'Ken H. Keller', regex=False)
final.data = final.data.str.replace('Simon. R. Barker', 'Simon R. Barker', regex=False)
final.data = final.data.str.replace('Adriana L Randall', 'Adriana L. Randall', regex=False)
final.data = final.data.str.replace('Moon Sung Keun', 'Moon Sung-Keun', regex=False)
final.data = final.data.str.replace('BD Wong', 'B.D. Wong', regex=False)
final.data = final.data.str.replace('Gustavo Sánchez Parra', 'Gustavo Sanchez Parra', regex=False)
final.data = final.data.str.replace('Soo Hyun Kim', 'Soo-hyun Kim', regex=False)
final.data = final.data.str.replace('Jean De-Rivieres', 'Jean de Rivieres', regex=False)
final.data = final.data.str.replace('Dong-hyun Kim', 'Dong Hyun Kim', regex=False)
final.data = final.data.str.replace('Hye Jin Jang', 'Hye-jin Jang', regex=False)
final.data = final.data.str.replace('Christopher St John', 'Christopher St. John', regex=False)
final.data = final.data.str.replace('Bjørn Lomborg', 'Bjorn Lomborg', regex=False)
final.data = final.data.str.replace('Young Min Kim', 'Young-min Kim', regex=False)
final.data = final.data.str.replace('Tom Goodman Hill', 'Tom Goodman-Hill', regex=False)
final.data = final.data.str.replace('Brian A Miller', 'Brian A. Miller', regex=False)
final.data = final.data.str.replace('Atsurô Watabe', 'Atsuro Watabe', regex=False)
final.data = final.data.str.replace('Hoe Jin Ha', 'Hoe-Jin Ha', regex=False)
final.data = final.data.str.replace('Young Ju Seo', 'Young-ju Seo', regex=False)
final.data = final.data.str.replace('Andrew Keenan Bolger', 'Andrew Keenan-Bolger', regex=False)
final.data = final.data.str.replace('Peter Henry Arnatsiaq', 'Peter-Henry Arnatsiaq', regex=False)
final.data = final.data.str.replace('Michael J Roberts', 'Michael J. Roberts', regex=False)
final.data = final.data.str.replace('Lucy Jo Hudson', 'Lucy-Jo Hudson', regex=False)
final.data = final.data.str.replace('Chi Kwong Cheung', 'Chi-Kwong Cheung', regex=False)
final.data = final.data.str.replace('Nick Cave & The Bad Seeds', 'Nick Cave & the Bad Seeds', regex=False)
final.data = final.data.str.replace('J.R.', 'JR', regex=False)
final.data = final.data.str.replace('David t Hyun', 'David T. Hyun', regex=False)
final.data = final.data.str.replace('Marie Therese Guirgis', 'Marie-Therese Guirgis', regex=False)
final.data = final.data.str.replace('Gustavo Sanchez Parra', 'Gustavo Sanchez-Parra', regex=False)
final.data = final.data.str.replace('Pierre Olivier Bardet', 'Pierre-Olivier Bardet', regex=False)

In [ ]:
## anything without a link attached to their person is an empty data value due to the way 
## the html was read so we're getting rid of those lines
final = final[final.link != 'https://www.metacritic.com/person?filter-options=movies'].reset_index(drop=True)

In [ ]:
## putting the film details dataframe into postgres
q_details = """
INSERT INTO movie_details
(movie_id, runtime, mpaa_rating, production_companies, genres, countries, languages)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
conn = pg.connect(**connection_args)
cur = conn.cursor()

In [ ]:
for row in movie_details.itertuples():
    cur.execute(q_details, (row[1:]))
conn.commit()
conn.close()

In [ ]:
## putting all the cast and crew credits into postgres
q_cast_crew = """
INSERT INTO cast_crew_credits (movie_id, label, person, link)
VALUES (%s, %s, %s, %s)"""

In [ ]:
conn = pg.connect(**connection_args)
cur = conn.cursor()
for row in final.itertuples():
    cur.execute(q, (row[1:]))
conn.commit()
conn.close()
final.to_csv('final-metacritic-df.csv', index=False)

In [ ]:
## putting metascore and userscore into postgres. table is setup to have
## the id successively added in 
q_score_scores = """
INSERT INTO metacritic_scores (title, release_date, link, mpaa, metascore, userscore)
VALUES (%s, %s, %s, %s, %s, %s)"""

In [ ]:
conn = pg.connect(**connection_args)
cur = conn.cursor()
for row in scores_and_dates.itertuples():
    cur.execute(q, (row[1:]))
conn.commit()
conn.close()
final.to_csv('final-metacritic-df.csv', index=False)
scores_and_dates.to_csv('metascore.csv', index=False)